In [ ]:
import glob

# json files from lableme
base_path = "/trunk2/yufei/summer24/EEBO_page_extraction/labelme_json"

all_json = glob.glob(base_path+"/*.json")
print("Num of the json files: ", len(all_json))

In [ ]:
import json

def get_polygon_baseline(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)
    
    polygons = []
    for shape in data['shapes']:
        if shape['label'] == 'page':
            polygon = {
                'polygon': shape['points'],
                'confidence': shape.get('confidence', 1),  # Default confidence to 1 if not present
                'channel': shape.get('channel', 'page')  
            }
            polygons.append(polygon)
    
    return polygons

# Example usage
# json_file_path = '/trunk2/yufei/summer24/EEBO_page_extraction/labelme_json/A00294_00005.000.001.json'
# result = get_polygon_baseline(json_file_path)
# print(result)


In [ ]:
import os
import shutil

########################################
# important: after adding new lableme json, update train_list.txt!!!!!!!!                                
########################################

def read_file_list(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    return [line.strip().replace('.tif', '.json').replace('/', '_') for line in lines]

def copy_image_files(file_list, source_image_directory, output_image_directory, json_directory):
    os.makedirs(output_image_directory, exist_ok=True)
    copied_count = 0
    for file_name in file_list:
        json_file_name = file_name
        original_file_name = file_name.replace('_', '/').replace('.json', '.tif')
        source_image_path = os.path.join(source_image_directory, original_file_name)
        destination_image_path = os.path.join(output_image_directory, file_name.replace('.json', '.tif'))
        json_file_path = os.path.join(json_directory, json_file_name)
        
        if os.path.exists(json_file_path) and os.path.exists(source_image_path) and os.path.isfile(source_image_path):
            shutil.copy2(source_image_path, destination_image_path)
            copied_count += 1
            print(f"Copied and renamed image {source_image_path} to {destination_image_path}")
        else:
            print(f"Image file {source_image_path} or JSON file {json_file_path} not found or is a directory!")
    
    return copied_count

def process_json_files(source_directory, train_list, validation_list, train_output_directory, validation_output_directory):
    # Ensure output directories exist
    os.makedirs(train_output_directory, exist_ok=True)
    os.makedirs(validation_output_directory, exist_ok=True)

    for filename in os.listdir(source_directory):
        if filename.endswith('.json'):
            source_file_path = os.path.join(source_directory, filename)

            if filename in train_list:
                output_directory = train_output_directory
            elif filename in validation_list:
                output_directory = validation_output_directory
            else:
                continue  # Skip files not in train or validation lists

            output_file_path = os.path.join(output_directory, filename)
            
            # Process JSON file
            polygons = get_polygon_baseline(source_file_path)
            
            # Write polygons to new JSON file
            with open(output_file_path, 'w') as output_file:
                json.dump(polygons, output_file, indent=4)
            
            print(f"Processed {filename} and saved to {output_file_path}")

# Example usage
source_directory = '/trunk2/yufei/summer24/EEBO_page_extraction/labelme_json'
train_list_path = '/trunk2/yufei/summer24/EEBO_page_extraction/train_list.txt'
validation_list_path = '/trunk2/yufei/summer24/EEBO_page_extraction/validation_list.txt'
train_output_json_directory = '/trunk2/yufei/summer24/EEBO_page_extraction/dataset/jsons/train'
validation_output_json_directory = '/trunk2/yufei/summer24/EEBO_page_extraction/dataset/jsons/validation'
train_output_image_directory = '/trunk2/yufei/summer24/EEBO_page_extraction/dataset/pictures/train'
validation_output_image_directory = '/trunk2/yufei/summer24/EEBO_page_extraction/dataset/pictures/validation'
source_image_directory = '/trunk/shared/pair_data'

# Read file lists
train_list = read_file_list(train_list_path)
validation_list = read_file_list(validation_list_path)

# Process JSON files and copy images
process_json_files(source_directory, train_list, validation_list, train_output_json_directory, validation_output_json_directory)

# Copy image files and count the total number of copied images
total_copied_images = 0
total_copied_images += copy_image_files(train_list, source_image_directory, train_output_image_directory, source_directory)
total_copied_images += copy_image_files(validation_list, source_image_directory, validation_output_image_directory, source_directory)

print(f"Total number of copied images: {total_copied_images}")

In [ ]:
import glob

# json files from lableme
base_path = "/trunk2/yufei/summer24/EEBO_page_extraction/dataset/jsons/train"

all_json = glob.glob(base_path+"/*.json")
print("Num of the json files: ", len(all_json))

In [ ]:
import glob

# json files from lableme
base_path = "/trunk2/yufei/summer24/EEBO_page_extraction/dataset/pictures/train"

all_json = glob.glob(base_path+"/*.tif")
print("Num of the json files: ", len(all_json))

In [ ]:
# import os
# import shutil

# def copy_and_rename_tif_files(source_directory, target_directory):
#     # Ensure the target directory exists
#     os.makedirs(target_directory, exist_ok=True)
    
#     copied_files_count = 0

#     # Traverse the source directory
#     for root, dirs, files in os.walk(source_directory):
#         for file in files:
#             if file.endswith('.tif'):
#                 # Construct the full file path
#                 source_file_path = os.path.join(root, file)
                
#                 # Create the new file name by replacing '/' with '_'
#                 relative_path = os.path.relpath(source_file_path, source_directory)
#                 new_file_name = relative_path.replace(os.sep, '_')
                
#                 # Construct the target file path
#                 target_file_path = os.path.join(target_directory, new_file_name)

#                 # Copy the file
#                 shutil.copy2(source_file_path, target_file_path)
#                 copied_files_count += 1
#                 print(f"Copied {source_file_path} to {target_file_path}")

#     # Print the total number of processed files
#     print(f"Total number of processed images: {copied_files_count}")

# # Example usage
# source_directory = '/trunk/shared/pair_data'
# target_directory = '/trunk3/shared/datasets/EEBO'

# copy_and_rename_tif_files(source_directory, target_directory)
